Readme
===========================
Program
--------------
 - provádí integraci z funkce 2 proměnných (s jistými omezeními - viz níže) přes oblast v \mathbb{R}^2
 - používá metodu konjugovaných gradientů pro nalezení zadané oblasti (je-li dána funkcí g, viz níže)
 - integraci provádí pomocí lichoběžníkového pravidla v obou proměnných
 - byl napsán v jazyce Python 3.7.1 z balíčku Anaconda, prostředí Jupyter Notebook
 - nebyl optimalizován, tudíž při požadavku na velkou přesnost může být velmi náročný na paměť (předchozí verze si řekla i o 3,6 GB paměti a 2 GB swapu, aktuální nebyla testována)
 - "blbuvzdornost" není implementovaná -> očekáváme, že uživatel má představu o požadované přesnosti

Očekávané vstupy:
------------------------------------------
 - funkce 2 proměnných, zadaná pomocí kartézských souřadnic, konečná na celé oblasti G \in \mathbb{R}^{2}, pomocí konečného počtu elementárních funkcí (které jazyk Python, potažmo balíček Numpy zná
 - nebo f-ce f(x,y) daná "vidličkou", viz příklad níže
 - konvexní oblast G zadaná jedním ze způsobů uvedených v Příkladu níže (či jinak, neváhejte vyzkoušet)

Nedostatky programu
------------------------------------------
 - používá lichoběžníkové pravidlo - nepřesné a náročné, jelikož rozdělujeme na 1D integrály, bylo by vhodnější užít Simpsona či Romberga
 - omezení pouze na klasickou míru \Phi(x) = x
 - oblast daná funkcí g -> nefunguje na cokoli hranatého, množiny zadané výčtem prvků, rozsahem souřadnic, Fourierovým rozvojem apod.
 - nefunguje na množiny jiné dimenze než 2 a ne-oblasti (např. křivky, body, celé R^2 apod.
 - omezení na elementární f-ce (nebere funkce zadané pomocí Taylorovy či Fourierovy řady, Besselovy f-ce,..)
 

In [39]:
#import knihoven
#=========================

import numpy as np

In [41]:
#vstup od uzivatele
#================================
#sem zadejte vasi funkci, jejiz integral chcete vypocitat
def f(x,y):
    if (abs(x)<=1):
        return 1.0
    else:
        return 0

#konvexni oblast zadavame ve tvaru g(x,y)<=0 - zde zadejte g(x,y)
def g(x,y):
    if(abs(x)<=2 and abs(y)<=2):
        return -1
    else:
        return 1

#pocatecni bod pro hledani oblasti
x0=0.0
y0=0.0

#konecna diference pro vypocet parc. derivaci
h=0.001

#kroky 
desc_step = 0.01
edge_step = 0.001
integrate_step = 0.01

#pozadovana presnost nalezeni minima oblasti - pokud oblast není příliš malá, tak je nepodstatné
desc_eps = 0.0001

#max. pocet iteraci
desc_nmax=50000
edge_nmax=50000

In [ ]:
#Příklady fungujících tvarů zadání:
#==================================

#Funkce
#----------------------------------

#def f(x,y): (konstantní funkce)
    #return 1.0
    
#def f(x,y): (elementární funkce)
    #return x**2 + np.exp(-x**2 - 3*y**2) + 2*np.sin(y)
    
#def f(x,y):  (fce zadaná "vidličkou")
    #if(abs(x)<2):
        #return 3
    #elif(abs(y<5)):
        #return 1
    #else:
        #return 0
        
#Oblasti
#-----------------------------------

#def g(x,y): (oblast daná jako vnitřek křivky, zde např. vnitřek elipsy)
    #return x**2 + y**2 -1
    
#def g(x,y): (oblast daná rozsahem souřadnic, zde např. čtverec)
    #if(abs(x)<=2 and abs(y) <=2):
        #return -1
    #else:
        #return 1
#Pozor - u takového zadání neexistuje derivace g(x,y) na celém R^2, 
#proto metoda konjug. gradientů nebude fungovat a je zapotřebí zadat 
#jako počáteční hodnoty x0,y0 body z této oblasti)

In [42]:
#parcialni derivace

def d_dx(f,x,y,h):
    return ((f(x+h,y)-f(x,y))/h)

def d_dy(f,x,y,h):
    return ((f(x,y+h)-f(x,y))/h)

In [43]:
#metoda konjugovanych gradientu pro hledani minim f-ci 2d
def gradient_descent_2d(f,d_dx,d_dy,x_0,y_0,step, nmax, eps, h):
    n = 0
    while n < nmax:
        x_1 = x_0
        y_1 = y_0
        x_0 = x_1 - step * d_dx(f, x_1, y_1, h)
        y_0 = y_1 - step * d_dy(f, x_1, y_1, h)
        if abs(x_0 - x_1) < eps and abs(y_0 - y_1) < eps:
            print("minimum found using", n, "iterations")
            return x_0, y_0
        n += 1
    raise RuntimeError("minimum not found using", nmax, "iterations")

In [44]:
#f-ce spoustejici integral v 2d oblasti z integrovani v 1d
def integrate_2d(f, g, x, y, integrate_step, ds):
    res=0
    x0=x
    y0=y
    prir0_hore = integrate_1d(f,g,x,y,integrate_step, 1, ds)[0]
    prir0_dole = integrate_1d(f,g,x,y,integrate_step, 0, ds)[0]
    while (g(x,y)<=0):
        x+=integrate_step
        
        
        [prir, y_max] = integrate_1d(f,g,x,y,integrate_step, 1, ds)
        res += ds * (prir + prir0_hore)
        
        prir0_hore = prir
        
        
        
        [prir, y_min] = integrate_1d(f,g,x,y,integrate_step, 0, ds)
        res += ds * (prir + prir0_dole)
        
        prir0_dole = prir
        
        
        y = 0.5*(y_min + y_max)
        
    x=x0-integrate_step
    y=y0
    
    prir0_hore = integrate_1d(f,g,x,y,integrate_step, 1, ds)[0]
    prir0_dole = integrate_1d(f,g,x,y,integrate_step, 0, ds)[0]
    while (g(x,y)<=0):
        x-=integrate_step
        
        [prir, y_max] = integrate_1d(f,g,x,y,integrate_step, 1, ds)
        res += ds * (prir + prir0_hore)
        
        prir0_hore = prir
        
        [prir, y_min] = integrate_1d(f,g,x,y,integrate_step, 0, ds)
        res += ds * (prir + prir0_dole)
        
        prir0_dole = prir
        
        y = 0.5*(y_min + y_max)
        
    return res

In [45]:
#jednodimenzionální integrace
def integrate_1d(f,g,x,y, step, way, mus):
    res = 0
    if(way==1):
        while(g(x,y)<=0):
            res += mus * (f(x,y) + f(x,y+step))
            y += step
            
        bord_y = y-step
        
    elif(way==0):
        while(g(x,y)<=0):
            res += mus * (f(x,y) + f(x,y-step))         
            y -=step
            
        bord_y = y+step
        
    else:
        raise RuntimeError('Unknown parameter.') 
        
    return [res, bord_y] 

In [46]:
def integral_2d(f,g,d_dx,d_dy,x0,y0,desc_step, edge_step, integrate_step, desc_nmax, edge_nmax, desc_eps,h):
    ds = integrate_step/2.0
    x_min, y_min = gradient_descent_2d(g,d_dx,d_dy,x0,y0, desc_step, desc_nmax, desc_eps, h)
    print("Found minimum!")
    
    if (g(x_min, y_min) > 0):
        raise RuntimeError("Failed to estimate given area to integrate on. Try lower desc_eps or better x0, y0.")
    else:
        print("Found minimum [",x_min, ",",y_min,"] is inside given area. Great, going to integrate.")
    
    
    result = integrate_2d(f, g, x_min, y_min, integrate_step, ds)
    
    print("V ramci zvolene presnosti je integral z dane funkce f priblizne")
    print(result)

In [47]:
integral_2d(f,g,d_dx,d_dy,x0,y0,desc_step, edge_step, integrate_step, desc_nmax, edge_nmax, desc_eps,h)

minimum found using 0 iterations
Found minimum!
Found minimum [ 0.0 , 0.0 ] is inside given area. Great, going to integrate.
V ramci zvolene presnosti je integral z dane funkce f priblizne
7.919999999999919
